## nn Approach



### Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

import os
from pathlib import Path

from pyMLaux import plot_history, evaluate_classification_result

2024-05-06 20:58:44.700281: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-06 20:58:44.775182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 20:58:44.775229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 20:58:44.776687: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 20:58:44.790723: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# read dynamic path
base_dir = Path(os.getcwd()) / "implementation"
data_dir = base_dir / "data/source/"
result_dir = base_dir / "data/results/"

### load & prepare dataset

the following code needs to be adapted for each protein-ligand complex individually

In [3]:
nn_data_raw_mdi = pd.read_csv(data_dir/"ACHE/ache_mdi.csv")
nn_data_raw_per = pd.read_csv(data_dir/"ACHE/ache_per.csv")
nn_data_raw = pd.read_csv(data_dir/"ACHE/ache.csv")

In [4]:
lookup = {'inactive':0,'active':1}

nn_data_per = {'data': np.array(nn_data_raw_per.iloc[:, 1:-1]),
             'target': np.array([lookup[y] for y in nn_data_raw_per.iloc[0:,-1]]),
             'feature_names': nn_data_raw_per.columns[1:-1],
             'target_names': ['inactive', 'active']}

nn_data_mdi = {'data': np.array(nn_data_raw_mdi.iloc[:, 1:-1]),
             'target': np.array([lookup[y] for y in nn_data_raw_mdi.iloc[0:,-1]]),
             'feature_names': nn_data_raw_mdi.columns[1:-1],
             'target_names': ['inactive', 'active']}

nn_data_base = {'data': np.array(nn_data_raw.iloc[:, 2:-1]),
             'target': np.array([lookup[y] for y in nn_data_raw.iloc[0:,-1]]),
             'feature_names': nn_data_raw.columns[2:-1],
             'target_names': ['inactive', 'active']}


split into train- and test-set

In [5]:
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(nn_data_base['data'], nn_data_base['target'],
                                                    test_size=0.3, random_state=4232)

X_train_mdi, X_test_mdi, y_train_mdi, y_test_mdi = train_test_split(nn_data_mdi['data'], nn_data_mdi['target'],
                                                    test_size=0.3, random_state=4232)

X_train_per, X_test_per, y_train_per, y_test_per = train_test_split(nn_data_per['data'], nn_data_per['target'],
                                                    test_size=0.3, random_state=4232)

### train and apply neural network

In [6]:
model_base = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(nn_data_base['data'].shape[1], )),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_base.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_per = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(nn_data_per['data'].shape[1], )),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_per.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_mdi = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(nn_data_mdi['data'].shape[1], )),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_mdi.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



2024-05-06 20:58:51.946604: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 20:58:52.041520: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 20:58:52.041647: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 20:58:52.051628: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 20:58:52.051903: I external/local_xla/xla/stream_executor

In [7]:
history_base = model_base.fit(x=X_train_base, y=y_train_base, epochs=150, batch_size=16, validation_split=0.2, verbose=2)
history_per = model_per.fit(x=X_train_per, y=y_train_per, epochs=150, batch_size=16, validation_split=0.2, verbose=2)
history_mdi = model_mdi.fit(x=X_train_mdi, y=y_train_mdi, epochs=150, batch_size=16, validation_split=0.2, verbose=2)


Epoch 1/150


2024-05-06 20:58:55.342910: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-06 20:58:57.020475: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f958c8ea7e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-06 20:58:57.020697: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-05-06 20:58:57.049484: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-06 20:58:57.150183: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1715021937.498027   15962 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


36/36 - 5s - loss: 0.7061 - accuracy: 0.5579 - val_loss: 0.6779 - val_accuracy: 0.5816 - 5s/epoch - 146ms/step
Epoch 2/150
36/36 - 1s - loss: 0.6641 - accuracy: 0.6168 - val_loss: 0.6533 - val_accuracy: 0.6099 - 621ms/epoch - 17ms/step
Epoch 3/150
36/36 - 1s - loss: 0.6338 - accuracy: 0.6471 - val_loss: 0.6355 - val_accuracy: 0.6454 - 585ms/epoch - 16ms/step
Epoch 4/150
36/36 - 1s - loss: 0.6100 - accuracy: 0.6791 - val_loss: 0.6205 - val_accuracy: 0.6383 - 683ms/epoch - 19ms/step
Epoch 5/150
36/36 - 1s - loss: 0.5902 - accuracy: 0.7077 - val_loss: 0.6081 - val_accuracy: 0.6596 - 707ms/epoch - 20ms/step
Epoch 6/150
36/36 - 1s - loss: 0.5744 - accuracy: 0.7237 - val_loss: 0.5976 - val_accuracy: 0.6738 - 719ms/epoch - 20ms/step
Epoch 7/150
36/36 - 1s - loss: 0.5593 - accuracy: 0.7398 - val_loss: 0.5881 - val_accuracy: 0.6809 - 692ms/epoch - 19ms/step
Epoch 8/150
36/36 - 1s - loss: 0.5463 - accuracy: 0.7683 - val_loss: 0.5794 - val_accuracy: 0.7163 - 650ms/epoch - 18ms/step
Epoch 9/150
36

In [12]:
df = pd.read_csv(result_dir/"ACHE/val/ache_val.csv",sep=",",index_col="Index")
df.loc[len(df["Name"])] = ["fe_rf_per_nn",0.7518]
df.to_csv(result_dir/"ACHE/val/ache_val.csv",sep=",",index="Index")

df = pd.read_csv(result_dir/"ACHE/val/ache_val.csv",sep=",",index_col="Index")
df.loc[len(df["Name"])] = ["fe_rf_mdi_nn",0.7589]
df.to_csv(result_dir/"ACHE/val/ache_val.csv",sep=",",index="Index")

Evaluate Testdata using model

In [8]:

pred_base = model_base.predict(X_test_base)
pred_per = model_per.predict(X_test_per)
pred_mdi = model_mdi.predict(X_test_mdi)

print("---------Base-Prediction----------")
evaluate_classification_result(y_test_base,pred_base,classes=nn_data_base["target_names"])
print("---------Permutation-FeatureSelection-Prediction----------")
evaluate_classification_result(y_test_per,pred_per,classes=nn_data_per["target_names"])
print("---------MDI-FeatureSelection-Prediction----------")
evaluate_classification_result(y_test_mdi,pred_mdi,classes=nn_data_mdi["target_names"])


10/10 [==============================] - 0s 2ms/step
---------Base-Prediction----------
[[137   0]
 [164   0]]


Class inactive:
    Sensitivity (TPR): 100.000% (137 of 137)
    Specificity (TNR):   0.000% (0 of 164)
    Precision:          45.515% (137 of 301)
    Neg. pred. value:      nan% (0 of 0)
Class active:
    Sensitivity (TPR):   0.000% (0 of 164)
    Specificity (TNR): 100.000% (137 of 137)
    Precision:             nan% (0 of 0)
    Neg. pred. value:   45.515% (137 of 301)

Overall accuracy:   45.515% (137 of 301)
Balanced accuracy:  50.000%
---------Permutation-FeatureSelection-Prediction----------
[[137   0]
 [164   0]]


Class inactive:
    Sensitivity (TPR): 100.000% (137 of 137)
    Specificity (TNR):   0.000% (0 of 164)
    Precision:          45.515% (137 of 301)
    Neg. pred. value:      nan% (0 of 0)
Class active:
    Sensitivity (TPR):   0.000% (0 of 164)
    Specificity (TNR): 100.000% (137 of 137)
    Precision:             nan% (0 of 0)
    Neg. pred. value: 

/root/.pyenv/versions/3.10.12/envs/activity_prediction/lib/python3.10/site-packages/pyMLaux/functions.py:163: RuntimeWarning: invalid value encountered in scalar divide
  print('    Neg. pred. value:  %7.3f%% (%d of %d)'%(100. * tn / (tn + fn) , tn, tn + fn))
/root/.pyenv/versions/3.10.12/envs/activity_prediction/lib/python3.10/site-packages/pyMLaux/functions.py:158: RuntimeWarning: invalid value encountered in scalar divide
  prec = tp / (tp + fp)
/root/.pyenv/versions/3.10.12/envs/activity_prediction/lib/python3.10/site-packages/pyMLaux/functions.py:163: RuntimeWarning: invalid value encountered in scalar divide
  print('    Neg. pred. value:  %7.3f%% (%d of %d)'%(100. * tn / (tn + fn) , tn, tn + fn))
/root/.pyenv/versions/3.10.12/envs/activity_prediction/lib/python3.10/site-packages/pyMLaux/functions.py:158: RuntimeWarning: invalid value encountered in scalar divide
  prec = tp / (tp + fp)
/root/.pyenv/versions/3.10.12/envs/activity_prediction/lib/python3.10/site-packages/pyMLaux/fu

array([[137,   0],
       [164,   0]])

In [9]:
pred_base = model_base.predict(X_test_base)
classes_base = [1 if i > 0.5  else 0 for i in pred_base]

pred_per = model_per.predict(X_test_per)
classes_per = [1 if i > 0.5  else 0 for i in pred_per]

pred_mdi = model_mdi.predict(X_test_mdi)
classes_mdi = [1 if i > 0.5  else 0 for i in pred_mdi]

10/10 [==============================] - 0s 3ms/step


In [10]:
result_df = pd.concat(
    [
        pd.DataFrame(columns=["INDEX"]),
        pd.DataFrame(columns=nn_data_raw_per.columns[1:-1]),
    ]
)

for i, row in enumerate(X_test_per):
    data = [i]
    data.extend(row)
    result_df.loc[len(result_df["INDEX"])] = data

result_df["LABEL"] = y_test_per
result_df["PRED"] = classes_per

result_df.to_csv(result_dir/"ACHE/fe_rf_per_nn.csv",encoding="utf-8")

In [11]:
result_df = pd.concat(
    [
        pd.DataFrame(columns=["INDEX"]),
        pd.DataFrame(columns=nn_data_raw_mdi.columns[1:-1]),
    ]
)

for i, row in enumerate(X_test_mdi):
    data = [i]
    data.extend(row)
    result_df.loc[len(result_df["INDEX"])] = data

result_df["LABEL"] = y_test_mdi
result_df["PRED"] = classes_mdi

result_df.to_csv(result_dir / "ACHE/fe_rf_mdi_nn.csv", encoding="utf-8")